<a href="https://colab.research.google.com/github/505719/M7DataSP/blob/main/HW02_FashionMNIST_Dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's build a simple neural network to classify images from the FashionMNIST dataset.

**1. Import Libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

*Checking for GPU Availability*

This code checks if a CUDA-enabled GPU is available and sets the `device` accordingly. If no GPU is available, it defaults to the CPU.

In [ ]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


**2. Data Preparation**

In [ ]:
# Define a transform to convert images to tensors
transform = transforms.ToTensor()

# Download and load the training data
train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

# Download and load the test data
test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)

**3. Neural Network Model**

In [ ]:
class DenseNet(nn.Module):
    def __init__(self): # co sa ma so sietou stat na zaciatku
        super(DenseNet, self).__init__()
        self.flatten = nn.Flatten() # z obrazka 28x28 pixelov urobi 1-rozmerne pole 1x(28*28)
        self.fc1 = nn.Linear(28 * 28, 128) # 1. vrstva, na vstupe je 28*28 pixelov, mame 128 neuronov
        self.fc2 = nn.Linear(128, 10) # 2. vrstva, 10 neuronov
        # dalsiu vrstvu by sme mohli pridat nasledovne:
        # self.fc2 = nn.Linear(128, 32)
        # self.fc3 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        logits = self.fc2(x)
        # logits = self.fc3(x)
        return logits

model = DenseNet().to(device)
model

DenseNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

**4. Loss & Optimizer**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1) # lr = learning rate

**5. Training loop**

In [ ]:
for epoch in range(10):  # Train for 10 epochs
    running_loss = 0.0
    for images, labels in train_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() # vezmeme parametry a odcitame learning rate * gradient

    print(f'Epoch [{epoch+1}/10], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/10], Loss: 0.6325
Epoch [2/10], Loss: 0.4431
Epoch [3/10], Loss: 0.3977
Epoch [4/10], Loss: 0.3709
Epoch [5/10], Loss: 0.3496
Epoch [6/10], Loss: 0.3350
Epoch [7/10], Loss: 0.3219
Epoch [8/10], Loss: 0.3116
Epoch [9/10], Loss: 0.3005
Epoch [10/10], Loss: 0.2930


**6. Evaluation on the test set**

In [ ]:
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for evaluation
    for images, labels in test_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 87.07%
